In [1]:
import numpy as np
import pandas as pd
import os
import glob

# !pip install ipywidgets
# Run before you create a new notebook with widgets
# !jupyter nbextension enable --py --sys-prefix widgetsnbextension
import ipywidgets as widgets

# !pip install plotly
import plotly as py
import plotly.graph_objs as go

py.offline.init_notebook_mode(connected=True)   # for offline mode use

from helpers import natural_sort

In [2]:
# merge all df into one
df = pd.read_csv('selectivity_vgg16.csv')
print(df.shape)
df.head()

(23040, 5)


,filter,category,selectivity,label,layer
0,0,0,-0.000099,furniture,block5_conv3
1,1,0,-0.000468,furniture,block5_conv3
2,2,0,-0.001807,furniture,block5_conv3
3,3,0,-0.000057,furniture,block5_conv3
4,4,0,-0.001234,furniture,block5_conv3


In [3]:
layout = go.Layout(
    title='<b>Category selectivity for filters</b>',
    yaxis=dict(
        title='<i>Selectivity</i>'
    ),
    xaxis=dict(
        title='<i>Filter</i>',
        tickangle=-90
    ),
    barmode='group',
)

def update_plot(categories, layer, sort_category, num_filters):
    
    """
    This function updates the plot everytime a widget is changed
    """
    d = df.loc[(df['filter'] >= num_filters[0]) & \
              (df['filter'] < num_filters[1]) & \
              (df['layer'] == layer), :].copy()
    d['filter'] = 'f_' + d['filter'].astype(str)
    
    list_ordering = [sort_category] + [x for x in d['label'].unique() if x != sort_category]
    cat_type = pd.api.types.CategoricalDtype(categories=list_ordering, ordered=True) 
    d['label'].astype(cat_type)
    d.sort_values(by=['label', 'selectivity'], ascending=[True, False], inplace=True)
    
    categories = [c for c in list_ordering if c in categories]
    
    data = []
    for c in categories: 
        group = d.loc[(d['label'] == c)] 
         
        trace1 = go.Bar(
            x=group['filter'],
            y=group['selectivity'],
            name='category {}'.format(c),
        )
        data.append(trace1)
        
    fig = go.Figure(data=data, layout=layout)
    py.offline.iplot(fig)
    
layers_ = natural_sort(df['layer'].unique())

layer = widgets.Dropdown(
    options=layers_, 
    value=layers_[0], 
    description='Layer:')

num_filters = widgets.IntRangeSlider(
    min=0, max=512, 
    value=(0, 512), 
    description='Filter range:')

sort_category = widgets.Dropdown(
    options=df['label'].unique().tolist(), 
    description='Sort by')

categories = widgets.SelectMultiple(
    options=df['label'].unique().tolist(), 
    value=df['label'].unique().tolist(),
    description='Categories')

g = widgets.interact(update_plot, categories=categories, layer=layer, 
                     sort_category=sort_category, num_filters=num_filters)

interactive(children=(SelectMultiple(description='Categories', index=(0, 1, 2, 3, 4, 5), options=('furniture', 'dog', 'iseeface', 'cat', 'portrait', 'car'), value=('furniture', 'dog', 'iseeface', 'cat', 'portrait', 'car')), Dropdown(description='Layer:', options=('block3_conv1', 'block3_conv2', 'block3_conv3', 'block4_conv1', 'block4_conv2', 'block4_conv3', 'block5_conv1', 'block5_conv2', 'block5_conv3'), value='block3_conv1'), Dropdown(description='Sort by', options=('furniture', 'dog', 'iseeface', 'cat', 'portrait', 'car'), value='furniture'), IntRangeSlider(value=(0, 512), description='Filter range:', max=512), Output()), _dom_classes=('widget-interact',))

In [4]:
layout = go.Layout(
    title='<b>Category selectivity for filters</b>',
    yaxis=dict(
        title='<i>Selectivity</i>'
    ),
    xaxis=dict(
        title='<i>Filter</i>',
        tickangle=-60,
        showticklabels=True,
        showgrid=True,
        showaxeslabels=True,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        )
    ),
    barmode='group',
    margin=dict(
    b= 20*8 # maxLabelLength * letterWidth
    ),
)



def update_top_plot(categories, layers, sort_category, num_top):
    
    """
    This function updates the plot everytime a widget is changed
    """
    # select top n filers per category per layer
    top_filters = (df.loc[(df['label'] == sort_category) & (df['layer'].isin(layers))]
             .groupby(by =['layer',  'filter'])
             .sum()
             .reset_index()
             .sort_values(['layer', 'selectivity'], ascending=[1,0])
             .groupby('layer').head(num_top)
         )[['layer', 'filter']]
    # find values in other categories per these combinations
    merged = df.merge(top_filters, how='outer', indicator=True)
    d = merged.copy()
    d = d[d['_merge'] == 'both']
    
    d['filter_layer'] = d['layer'] + '_' + d['filter'].astype(str)
    list_ordering = [sort_category] + [x for x in d['label'].unique() if x != sort_category]
    cat_type = pd.api.types.CategoricalDtype(categories=list_ordering, ordered=True)
    d['label'].astype(cat_type)
    d.sort_values(by=['label', 'selectivity'], ascending=[True, False], inplace=True)
    categories = [c for c in list_ordering if c in categories]
    
    data = []
    for c in categories: 
        group = d.loc[(d['label'] == c)] 
        trace1 = go.Bar(
            x=group['filter_layer'],
            y=group['selectivity'],
            name='{}'.format(c),
        )
        data.append(trace1)
        
    fig = go.Figure(data=data, layout=layout)
    py.offline.iplot(fig)
    
layers_ = natural_sort(df['layer'].unique())

layers = widgets.SelectMultiple(
    options=layers_, 
    value=(layers_),
    description='Layers:')

num_top = widgets.IntSlider(
    min=1, max=10, 
    value=1, 
    description='Top filters:')

sort_category = widgets.Dropdown(
    options=df['label'].unique().tolist(), 
    description='Sort by')

categories = widgets.SelectMultiple(
    options=df['label'].unique().tolist(), 
    value=(df['label'].unique()[0],),
    description='Categories')

g = widgets.interact(update_top_plot, categories=categories, layers=layers, 
                     sort_category=sort_category, num_top=num_top)

interactive(children=(SelectMultiple(description='Categories', index=(0,), options=('furniture', 'dog', 'iseeface', 'cat', 'portrait', 'car'), value=('furniture',)), SelectMultiple(description='Layers:', index=(0, 1, 2, 3, 4, 5, 6, 7, 8), options=('block3_conv1', 'block3_conv2', 'block3_conv3', 'block4_conv1', 'block4_conv2', 'block4_conv3', 'block5_conv1', 'block5_conv2', 'block5_conv3'), value=('block3_conv1', 'block3_conv2', 'block3_conv3', 'block4_conv1', 'block4_conv2', 'block4_conv3', 'block5_conv1', 'block5_conv2', 'block5_conv3')), Dropdown(description='Sort by', options=('furniture', 'dog', 'iseeface', 'cat', 'portrait', 'car'), value='furniture'), IntSlider(value=1, description='Top filters:', max=10, min=1), Output()), _dom_classes=('widget-interact',))